In [1]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/kdephys/kdephys/plot/acr_plots.mplstyle')

import plotly.express as px
from scipy.ndimage import gaussian_filter1d

import kaleido
save_base = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/plots_presentations_etc/temp'
from kdephys.utils.spectral import bands

/home/kdriessen/miniconda3/envs/acr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def bp_melt(bp, bp_def=bands):
    """Melts a bandpower set to long-form.

    Parameters:
    -----------
    bp_def: bandpower dictionary, supplied automatically from kdephys.utils.spectral
    """

    bp_melt = pd.melt(
        bp,
        id_vars=["datetime", "channel", "store"],
        value_vars=list(bp_def.keys()),
    )
    bp_melt.columns = ["datetime", "channel", "store", "Band", "Bandpower"]
    return bp_melt

In [3]:
def smooth_1d(array, sigma=14, fs=0.5, axis=0, truncate=8):
    return gaussian_filter1d(array, sigma=sigma*fs, axis=axis, truncate=truncate)

In [4]:
def smooth_whole_bp_set(ds, sigma=14, fs=0.5, axis=0, truncate=8):
    for da_name, da in ds.items():
        data = da.values
        smoothed_data = np.apply_along_axis(smooth_1d, 1, da.data)
        da.values = smoothed_data
        ds[da_name] = da
    return ds

In [13]:
subject = 'ACR_26'
recording = 'swi'
bp_xr = acr.io.load_concat_bandpower(subject, [recording], stores=['NNXo', 'NNXr'], hypno=False, exclude_bad_channels=False)
bp_xr_smooth = smooth_whole_bp_set(bp_xr, sigma=14, fs=0.5, axis=0, truncate=8)
bp = bp_xr.to_dataframe().reset_index()
bp_sm = bp_xr_smooth.to_dataframe().reset_index()

In [14]:
bp_melted = bp_melt(bp)
medians = bp_melted.groupby(['store', 'channel', 'Band']).Bandpower.median()
medians = medians.to_frame().reset_index()

In [15]:
# LINE PLOT
# ---------------
band = 'delta'
fig = px.line(bp_sm.ts(0, 43200), x='datetime', y=band, title=f'{subject} | {recording} | RAW {band}', color='store', facet_row='channel', facet_row_spacing=0.01, width=2400, height=2000, color_discrete_sequence=['royalblue', 'darkred'], range_y=[0, 300000], template='plotly_white')
fig.update_layout(
    font_color="black",
    font_size=18,
    title_font_size=28,
)
#fig.show()
fig.write_image(f'{save_base}/{subject}_{recording}_RAW-{band}.png', engine="kaleido")

In [18]:
acr.stim.stim_bookends('ACR_26', 'swisin2')

(Timestamp('2023-11-02 13:29:59.961133080'),
 Timestamp('2023-11-02 14:13:34.040621080'))

In [21]:
acr.utils.dt_to_tdt(subject, 'swisin2', '2023-11-02 13:29:59.961133080')

16029.961134

In [16]:
# BAR PLOT of MEDIANS
# -----------------------
cat_ord={"Band": bands.keys()}
title=f'{subject} | {recording} | Median Bandpower'
fbar = px.bar(medians, x='channel', y='Bandpower', color='store', facet_row='Band', barmode='group', facet_row_spacing=0.01, category_orders=cat_ord, width=1800, height=1800, color_discrete_sequence=['royalblue', 'darkred'], template='plotly_white', title=title)
fbar.update_yaxes(matches=None)
fbar.update_layout(
    font_color="black",
    font_size=18,
    title_font_size=28,
)
fbar.show()
fbar.write_image(f'{save_base}/{subject}_{recording}_RAW-median-bandpower.png', engine="kaleido")